# CityBikes

## 1. Send a request to CityBikes for the city of your choice. 

### I have written two functions 
1. get_city_id : To get city id from API city_bikes_url which takes "city" name as input.
                 I have collected data for Paris city.
                 
2. get_stations : To get station details in given city (Paris) by providing city_id as                       input obtained from first function.

In [1]:
import requests
import pandas as pd

def get_city_id(city):
    # Make a request to get data from the CityBikes API
    city_bikes_url = "https://api.citybik.es/v2/networks"  # URL for getting the list of available networks
    response = requests.get(city_bikes_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        city_bikes_data = response.json()
        city_id = None
        
        for network in city_bikes_data['networks']:
            if network['location']['city'] == city:
                city_id = network['id']
                break

        if city_id:
            return city_id
        else:
            print(f"No data available for the city: {city}")
            return None
    else:
        print(f"Error: {response.status_code}")
        return None


def get_stations(city_id):
    url = f'https://api.citybik.es/v2/networks/{city_id}' 
    # Make the API request and handle potential errors
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad requests

        # Check if the response content is not empty
        if response.content:
            data = response.json()
            stations = data['network']['stations']
            station_data = []

            for station in stations:
                station_info = {
                    'Name': station['name'],
                    'Latitude': station['latitude'],
                    'Longitude': station['longitude'],
                    'Bikes Available': station['free_bikes'],
                    'Slots': station['extra']['slots'],
                    'Empty Slots': station['empty_slots'],
                    'Banking': station['extra']['banking'],
                    'Ebikes': station['extra']['ebikes'],
                }
                station_data.append(station_info)

            # Create a DataFrame from the extracted data
            df = pd.DataFrame(station_data)
            return df
        else:
            print("No data available for the specified city.")
    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Error Connecting: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"Request Exception: {err}")
    except ValueError as e:
        print(f"Error parsing JSON: {e}")


city_name = "Paris"

# Get the city ID
city_id = get_city_id(city_name)

# Fetch bike station data using the city ID
if city_id:
    stations_df = get_stations(city_id)
else:
    print("City ID not found. Cannot fetch bike station data.")

## 2.Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

#### I have intentionally selected all values as I needed those further in the project

## 3.Put your parsed results into a DataFrame.

#### stations_df = get_stations(city_id)